In [ ]:
import sys
import os
import argparse
import datetime
import numpy as np

# import importlib
import traceback

from multiprocessing import Pool

# from importlib.import("vendor.BirdNET-Analyzer.analyze")
# from vendor.BirdNET_Analyzer import analyze
import analyze
import model
import os

import numpy as np

import warnings

warnings.filterwarnings("ignore")
# import tflite2onnx
# import tf2onnx
import config as cfg
import tensorflow as tf

In [ ]:
imported_model = tf.saved_model.load("checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/")

In [ ]:
imported_model.basic.input_signature

In [ ]:
imported_model.basic.input_signature

In [ ]:
imported_model.basic.pretty_printed_concrete_signatures()

In [ ]:
tflite_path = "checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model_FP16.tflite"
onnyx_path = "example/"

In [ ]:
def printTensors(pb_file):
    # read pb into graph_def

    with tf.io.gfile.GFile(pb_file, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    # import graph_def
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)

    # print operations
    for op in graph.get_operations():
        print(op.name)

In [ ]:
path_pb = "checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/"

In [ ]:
# printTensors("checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/saved_model.pb")

In [ ]:
# tf2onnx.convert.from_tflite(tflite_path=tflite_path, output_path= onnyx_path)

In [ ]:
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat
from tensorflow.core.protobuf import saved_model_pb2
with tf.compat.v1.Session(graph = tf.Graph()) as sess:
    model_filename = "checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/saved_model.pb"
    with gfile.FastGFile(model_filename, 'rb') as f:

            data = compat.as_bytes(f.read())
            sm = saved_model_pb2.SavedModel()
            sm.ParseFromString(data)
            #print(sm)
            if 1 != len(sm.meta_graphs):
                    print('More than one graph found. Not sure which to write')
                    sys.exit(1)

            graph_def = tf.compat.v1.GraphDef()\
            g_in = tf.import_graph_def(sm.meta_graphs[0].graph_def)
           
            #test = [n.name + '=>' + n.op for n in graph_def.node]
            
    with graph.as_default():


            saver1 = tf.compat.v1.train.import_meta_graph(sm.meta_graphs[0])


#            #saver1.restore(sess, save_path='model')
 
       
#             for v in tf.compat.v1.global_variables():
#                     print(v.name)

In [ ]:
sm.meta_graphs

In [ ]:
print(graph.get_operations())